# Wrangling Text

When you think of data cleaning, one task that probably comes to mind is wrangling text. After all, when people enter data on a form or different formatting conventions are appended together, you will likely find yourself standardizing the data and trying to make it consistent. You will also seek values that were lost in translation and are unusable. 

In this section we will cover a variety of techniques to wrangle text and perform tasks like finding, replacing, and splitting values. Along the way, we will learn some regular expressions to perform pattern recognition in these tasks. 

First let's bring in our dependencies, and look at this dataset from Github. Notice how we have some contact information as well as a log of IP address of different users. We are going to learn how to perform some common text operations to clean this dataset and enforce some consistency. 

In [1]:
import pandas as pd 
import numpy as np 

url = 'https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/unprocessed/contacts.csv'
df = pd.read_csv(url)

df

,First Name,Last Name,Email,Phone,Occupation,IP_ADDRESS
0,Tony,Reed,tony.reed@randatmail.com,990-788-3109,Electrician,124.56.176.35
1,Vanessa,Hawkins,v.hawkins@randatmail.com,219-571-0235,Social Worker,90.22.105.47
2,Eleanor,Anderson,e.anderson@randatmail.com,584-199-3924,Teacher,122.112.123.171
3,Maya,Mitchell,m.mitchell@randatmail.com,145-281-1156,Programmer,206.38.233.227
4,Melissa,Riley,m.riley@randatmail.com,(274)-764-0265,Interior Designer,137.252.6.22
5,Sofia,Richards,s.richards@randatmail.com,136-202-9861,Mathematician,229.194.40.223
6,Maya,Holmes,m.holmes@randatmail.com,617-127-2752,Cook,232.214.86.41
7,Melanie,Murphy,m.murphy@outlook.com,6236136485,Driver,98.22.71.90
8,Gianna,Allen,g.allen@randatmail.com,732-327-0765,Physicist,84.51.154.29
9,Cadie,Kelley,c.kelley@randatmail.com,873-980-1152,Interpreter,174.186.65.94


These are the common string operations in Pandas we can use. Note that these typically accept a regular expression as a pattern, and we will cover this. 

| Function   | Description                                                                 |
|------------|-----------------------------------------------------------------------------|
| `count()`    | Counts the number of instances in a pattern                                 |
| `contains()` | Returns a boolean True/False indicating whether a string contains a pattern |
| `replace()`  | Replaces the found patterns in a string with another specified string.      |
| `fullmatch()`    | Determines if the entire string matches the pattern                         |
| `split()`    | Splits a string into separate strings using the pattern as the separator    |
| `extract()`  | Finds all occurrences of a pattern and packages them into columns           |
| `findall()`  | Finds all occurrences of a pattern and packages them into a list            |

But first, we will need to cover a few basics with regular expressions. 

## Regular Expression Basics

If you ever have used wildcards to search for text patterns, regular expressions are similar. **Regular expressions** are a special programming language specifically for matching complex text patterns. They allow matching, splitting, and replacing text based on a standardized pattern syntax. You can find them implemented in hundreds of platforms including Python, Java, and SQL. Even IDE's and text editors will allow you to search text using regular expressions such as VSCode, PyCharm, and Notepad++. They are so useful that Pandas makes them the default pattern convention for many of its aforementioned string methods. 

We are going to learn just enough about regular expressions to get through this notebook.

> You can refer to Python's documentation on the `re` package here: https://docs.python.org/3/library/re.html. For a more thorough walkthrough on regular expressions, check out my article with O'Reilly: https://www.oreilly.com/content/an-introduction-to-regular-expressions/

Let's first just use plain Python's `re` library which implements regular expressions. We are going to test our regular expressions with the `fullmatch()` function, and wrap it up in a function called `regex_match()` that will simply print whether the pattern matches the string. It will also do some convenient font color formatting in the output. 

In [6]:
import re

def red(str): 
    return '\033[91m' + str + '\033[0m'

def green(str): 
    return '\033[92m' + str + '\033[0m'

def regex_match(string, pattern):
    result = re.fullmatch(pattern=pattern, string=string)

    if result:
        print(f"{green(string)} Matches {green(pattern)}")
    else:
        print(f"{red(string)} Doesn't Match {red(pattern)}")

To match a single uppercase alphabetic character, use the character range `[A-Z]` as a placeholder for a single character. Note how it is case senstive and you can also define arbitrary ranges of letters. 

In [8]:
regex_match("C", "[A-Z]") # Match
regex_match("F", "[A-C]") # Doesn't Match
regex_match("3", "[A-Z]") # Doesn't Match 
regex_match("c", "[A-Z]") # Doesn't Match 
regex_match("-", "[A-Z]") # Doesn't Match 

C Matches [A-Z]
F Doesn't Match [A-C]
3 Doesn't Match [A-Z]
c Doesn't Match [A-Z]
- Doesn't Match [A-Z]


To match both uppercase and lowercase letters, use `[A-Za-z]`. 

In [13]:
regex_match("C", "[A-ZA-z]") # Match
regex_match("c", "[A-Za-z]") # Matches
regex_match("3", "[A-Za-z]") # Doesn't Match 

C Matches [A-ZA-z]
c Matches [A-Za-z]
3 Doesn't Match [A-Za-z]


We can also use `[0-9]` to specify a valid digit 0-9, or any arbitrary range of a single digit. 

In [16]:
regex_match("9", "[0-9]") # Match
regex_match("c", "[A-Za-z0-9]") # Match
regex_match("9", "[3-6]") # Doesn't Match
regex_match("C", "[0-9]") # Doesn't Match

9 Matches [0-9]
c Matches [A-Za-z0-9]
9 Doesn't Match [3-6]
C Doesn't Match [0-9]


You can also specify a set of letters, digits and characters. Below we only qualify the characters A, C, F, 2, 8, or 9. 

In [19]:
regex_match("9", "[ACF289]") # Match
regex_match("C", "[ACF289]") # Match
regex_match("7", "[ACF289]") # Doesn't Match
regex_match("G", "[ACF289]") # Doesn't Match

9 Matches [ACF289]
C Matches [ACF289]
7 Doesn't Match [ACF289]
G Doesn't Match [ACF289]


Letters and digits outside a character range `[ ]` are literally treated as letters and digits in regular expressions. They will match only those values. 

In [22]:
regex_match("Texas", "Texas") # Match
regex_match("Texas", "Arizona") # Doesn't Match 
regex_match("Texas", "TEXAS") # Doesn't Match 

Texas Matches Texas
Texas Doesn't Match Arizona
Texas Doesn't Match TEXAS


If you want to match 3 uppercase alphabetic letters, either write `[A-Z]` three times or put `{3}` repetitions next to the character range.  You can also use `{2,3}` to specify a minimum of 2 repetitions and a maximum of `3`. 

In [25]:
regex_match("AEH", "[A-Z][A-Z][A-Z]") # Match
regex_match("AFH", "[A-Z]{3}") # Match
regex_match("AFH", "[A-Z]{2,3}") # Match
regex_match("AF", "[A-Z]{2,3}") # Match
regex_match("A9H", "[A-Z]{2,3}") # Doesn't Match

AEH Matches [A-Z][A-Z][A-Z]
AFH Matches [A-Z]{3}
AFH Matches [A-Z]{2,3}
AF Matches [A-Z]{2,3}
A9H Doesn't Match [A-Z]{2,3}


If you want to match one or more instances of a pattern, put a `+` next to it. For example, `[A-Z]+` will match 1 or more alphabetic uppercase characters.  

In [28]:
regex_match("AEH", "[A-Z]+") # Match
regex_match("AEHSDHHHNHEHHBV", "[A-Z]+") # Match
regex_match("93572", "[0-9]+") # Match
regex_match("AEHSDHHHNHEHHBV", "[A-Z0-9]+") # Match
regex_match("93572", "[A-Z]+") # Doesn't Match
regex_match("AEHSDHHHNHEHHBV", "[0-9]+") # Doesn't Match

AEH Matches [A-Z]+
AEHSDHHHNHEHHBV Matches [A-Z]+
93572 Matches [0-9]+
AEHSDHHHNHEHHBV Matches [A-Z0-9]+
93572 Doesn't Match [A-Z]+
AEHSDHHHNHEHHBV Doesn't Match [0-9]+


Another helpful quantifier is the `?` which matchs 0 or 1 instances of a pattern. For example, we can use it to specify an optional digit in front of two uppercase letters. 

In [31]:
regex_match("2GH", "[0-9]?[A-Z]{2}") # Match
regex_match("GH", "[0-9]?[A-Z]{2}") # Match
regex_match("2H", "[0-9]?[A-Z]{2}") # No Match
regex_match("22H", "[0-9]?[A-Z]{2}") # No Match

2GH Matches [0-9]?[A-Z]{2}
GH Matches [0-9]?[A-Z]{2}
2H Doesn't Match [0-9]?[A-Z]{2}
22H Doesn't Match [0-9]?[A-Z]{2}


The dot `.` represents a wildcard character, matching any single character including non-alphanumeric characters like punctuation and symbols. If you intend to match a literal dot, use an escape slash in front of it `\.`. 

With a wildcard character, you can also put a quantifier like `{3}` or `+` after it to specify 3 characters or one or more characters respectively.

In [34]:
regex_match("A#H", "...") # Match
regex_match("A#H", ".{3}") # Match 
regex_match("A#H", ".+") # Match
regex_match("AH", ".{3}") # Doesn't Match

A#H Matches ...
A#H Matches .{3}
A#H Matches .+
AH Doesn't Match .{3}


Finally, the last operator we need to know is grouping up parantheses `()` as well as the alternator `|`. If I want to only match airport connections from `ABQ` or `DAL` to `HOU` or `PHX`, I could express that with `(ABQ|DAL)-(HOU|PHX)`.  

In [37]:
regex_match("ABQ", "(ABQ|DAL)") # Match 
regex_match("ABQ-HOU", "(ABQ|DAL)-(HOU|PHX)") # Match 
regex_match("DAL-HOU", "(ABQ|DAL)-(HOU|PHX)") # Match 
regex_match("DAL-PHX", "(ABQ|DAL)-(HOU|PHX)") # Match 
regex_match("PHX-DAL", "(ABQ|DAL)-(HOU|PHX)") # Doesn't Match 
regex_match("MDW-DAL", "(ABQ|DAL)-(HOU|PHX)") # Doesn't Match 


ABQ Matches (ABQ|DAL)
ABQ-HOU Matches (ABQ|DAL)-(HOU|PHX)
DAL-HOU Matches (ABQ|DAL)-(HOU|PHX)
DAL-PHX Matches (ABQ|DAL)-(HOU|PHX)
PHX-DAL Doesn't Match (ABQ|DAL)-(HOU|PHX)
MDW-DAL Doesn't Match (ABQ|DAL)-(HOU|PHX)


## Partial String Matches

Let's say we want to find all records with an `Email` containing a domain of `outlook.com`. This is easy enough using the `contains()` function under the `str` property. Note that the pattern string is treated as a regular expression so we need to escape the dot `.` with a backslash `\.`. Otherwise, it will be treated as a wildcard.

In [41]:
df['Email'].str.contains('outlook\.com', regex=True)

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\User\AppData\Local\Temp\ipykernel_1348\975412174.py:1: SyntaxWarning: invalid escape sequence '\.'
  df['Email'].str.contains('outlook\.com', regex=True)


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7      True
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
17    False
18      NaN
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
Name: Email, dtype: object

Since one of the values for email is `NaN`, we will need to handle it if we are to use this as a filtering mask. We can do that by passing `na = False` to the `contains()` function. This will cause missing values to be treated as `False`. 

In [44]:
df[df['Email'].str.contains('outlook\.com', regex=True, na=False)]

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\User\AppData\Local\Temp\ipykernel_1348\692794325.py:1: SyntaxWarning: invalid escape sequence '\.'
  df[df['Email'].str.contains('outlook\.com', regex=True, na=False)]


,First Name,Last Name,Email,Phone,Occupation,IP_ADDRESS
7,Melanie,Murphy,m.murphy@outlook.com,6236136485,Driver,98.22.71.90
16,Clark,Wright,c.wright@outlook.com,511-617-4034,Mathematician,154.155.237.103


## Full String Matches

Let's say we want to hunt down invalid IP addresses. While we can [get wildly specific and elaborate with ipv4 patterns](https://stackoverflow.com/questions/5284147/validating-ipv4-addresses-with-regexp) let's keep it simple. 

Below is a simplistic regular exression to match an IP address. We use the `fullmatch()` to qualify the IP address string in full.

In [48]:
ipAddressRegex = r'[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}'

df['IP_ADDRESS'].str.fullmatch(ipAddressRegex)

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27    False
28     True
29     True
30     True
31     True
32     True
33     True
34     True
35     True
36     True
37     True
38     True
39     True
40     True
41     True
42     True
43     True
44     True
45     True
46     True
47     True
48     True
49     True
Name: IP_ADDRESS, dtype: bool

> Typically, you only need to make your regular expression as specific enough to capture what you're looking for in the data. If you do not know your data well, you will want to err on being more specific. 

Let's use to qualify IP addresses that don't match in a condition. Sure enough, we have one broken IP address that exceeds 4 digits between the `.` separators.

In [51]:
df[df['IP_ADDRESS'].str.fullmatch(ipAddressRegex) == False]

,First Name,Last Name,Email,Phone,Occupation,IP_ADDRESS
27,Catherine,Andrews,c.andrews@randatmail.com,869-530-9746,Producer,200.80008.94.7002


Here's another example finding invalid US phone numbers. Note how we qualify the first 3 digits, then the next 3, and then then final 4 digits. Variants that may or may not contain hypens `-`, parantheses for area code `( )`, and spaces. Sure enough we find three broken phone numbers.

In [54]:
df[df['Phone'].str.fullmatch(r"\(?[0-9]{3}\)?[ -]?[0-9]{3}[ -]?[0-9]{4}") == False]

,First Name,Last Name,Email,Phone,Occupation,IP_ADDRESS
23,Natalie,Holmes,n.holmes@randatmail.com,359-395-67635,Aeroplane Pilot,52.248.52.87
26,Maria,Fowler,m.fowler@gmail.com,236-45-9135,Archeologist,183.213.33.49
29,Adelaide,Farrell,a.farrell@randatmail.com,27-114-9352,Composer,232.99.127.47


Let's go ahead and only include rows in our dataframe that have valid phone numbers and IP addresses. 

In [57]:
df = df[df['Phone'].str.fullmatch(r"\(?[0-9]{3}\)?[ -]?[0-9]{3}[ -]?[0-9]{4}")]

df = df[df['IP_ADDRESS'].str.fullmatch(ipAddressRegex)]

df

,First Name,Last Name,Email,Phone,Occupation,IP_ADDRESS
0,Tony,Reed,tony.reed@randatmail.com,990-788-3109,Electrician,124.56.176.35
1,Vanessa,Hawkins,v.hawkins@randatmail.com,219-571-0235,Social Worker,90.22.105.47
2,Eleanor,Anderson,e.anderson@randatmail.com,584-199-3924,Teacher,122.112.123.171
3,Maya,Mitchell,m.mitchell@randatmail.com,145-281-1156,Programmer,206.38.233.227
4,Melissa,Riley,m.riley@randatmail.com,(274)-764-0265,Interior Designer,137.252.6.22
5,Sofia,Richards,s.richards@randatmail.com,136-202-9861,Mathematician,229.194.40.223
6,Maya,Holmes,m.holmes@randatmail.com,617-127-2752,Cook,232.214.86.41
7,Melanie,Murphy,m.murphy@outlook.com,6236136485,Driver,98.22.71.90
8,Gianna,Allen,g.allen@randatmail.com,732-327-0765,Physicist,84.51.154.29
9,Cadie,Kelley,c.kelley@randatmail.com,873-980-1152,Interpreter,174.186.65.94


Finally, let's identify all invalid email addresses. An email needs to have a series of alphanumeric characters (with some allowable symbols like dot `.`), followed by the `@` symbol, then the domain. We will also treat `na` as false to also capture missing email addresses.

In [60]:
df[df['Email'].str.fullmatch(r'[.A-Za-z0-9]+@[A-Za-z0-9]+\.[A-Za-z]+', na=False) == False]

,First Name,Last Name,Email,Phone,Occupation,IP_ADDRESS
18,Oliver,Williams,NaN,210-702-2551,Florist,106.7.15.18
36,Lily,Robinson,l.robinson@,458-564-1023,Actor,70.189.66.201


So we find two email addresses that are missing or broken. Lily's email is missing a domain! We will remove those two instances from the dataframe. 

In [63]:
df = df[df['Email'].str.fullmatch(r'[.A-Za-z0-9]+@[A-Za-z0-9]+\.[A-Za-z]+', na=False)]

df

,First Name,Last Name,Email,Phone,Occupation,IP_ADDRESS
0,Tony,Reed,tony.reed@randatmail.com,990-788-3109,Electrician,124.56.176.35
1,Vanessa,Hawkins,v.hawkins@randatmail.com,219-571-0235,Social Worker,90.22.105.47
2,Eleanor,Anderson,e.anderson@randatmail.com,584-199-3924,Teacher,122.112.123.171
3,Maya,Mitchell,m.mitchell@randatmail.com,145-281-1156,Programmer,206.38.233.227
4,Melissa,Riley,m.riley@randatmail.com,(274)-764-0265,Interior Designer,137.252.6.22
5,Sofia,Richards,s.richards@randatmail.com,136-202-9861,Mathematician,229.194.40.223
6,Maya,Holmes,m.holmes@randatmail.com,617-127-2752,Cook,232.214.86.41
7,Melanie,Murphy,m.murphy@outlook.com,6236136485,Driver,98.22.71.90
8,Gianna,Allen,g.allen@randatmail.com,732-327-0765,Physicist,84.51.154.29
9,Cadie,Kelley,c.kelley@randatmail.com,873-980-1152,Interpreter,174.186.65.94


## Finding All Matches

We can also use `findall()` to look for all partial matches of a regular expression and return them as a series. Below we extract all the email domains from the `Email` column.

In [67]:
df['Email'].str.findall(r'[A-Za-z0-9]+\.[A-Za-z]{3}$')

0     [randatmail.com]
1     [randatmail.com]
2     [randatmail.com]
3     [randatmail.com]
4     [randatmail.com]
5     [randatmail.com]
6     [randatmail.com]
7        [outlook.com]
8     [randatmail.com]
9     [randatmail.com]
10    [randatmail.com]
11    [randatmail.com]
12    [randatmail.com]
13    [randatmail.com]
14    [randatmail.com]
15    [randatmail.com]
16       [outlook.com]
17    [hamiltonco.org]
19    [randatmail.com]
20     [protomail.com]
21    [randatmail.com]
22    [randatmail.com]
24    [randatmail.com]
25    [randatmail.com]
28    [randatmail.com]
30    [randatmail.com]
31    [randatmail.com]
32    [randatmail.com]
33    [randatmail.com]
34    [randatmail.com]
35    [randatmail.com]
37    [randatmail.com]
38         [gmail.com]
39    [randatmail.com]
40    [randatmail.com]
41    [randatmail.com]
42    [randatmail.com]
43    [randatmail.com]
44    [randatmail.com]
45         [gmail.com]
46    [randatmail.com]
47    [randatmail.com]
48         [gmail.com]
49    [rand

If we wanted to gather the unique domains, we can join the "lists" of single items into a string and then qualify the unique values. 

In [70]:
df['Email'].str.findall(r'[A-Za-z0-9]+\.[A-Za-z]{3}$').str.join("").unique()

array(['randatmail.com', 'outlook.com', 'hamiltonco.org', 'protomail.com',
       'gmail.com'], dtype=object)

## Replacing Matches

Let's say we want to clean up phone numbers by removing any extraneous dashes `-`, parantheses `()`, and spaces ` `. We can do that by using a regular expression character set `[- ()]`. Note we have to make the dash `-` the first character so it doesn't get confused as a range operator. We also throw a space ` ` in there too so we capture spaces.

In [73]:
df['Phone'].str.replace(r"[- ()]", "", regex=True)

0     9907883109
1     2195710235
2     5841993924
3     1452811156
4     2747640265
5     1362029861
6     6171272752
7     6236136485
8     7323270765
9     8739801152
10    5856246547
11    2168831869
12    8850854183
13    8597827705
14    1766430467
15    8030978506
16    5116174034
17    1447139625
19    9291156383
20    5862292941
21    4986077405
22    4963633460
24    8029819045
25    3884847766
28    8955507674
30    9453781710
31    5462547930
32    9717655942
33    1989382968
34    6709188668
35    8234619936
37    2136398827
38    9302296062
39    3040200448
40    3610492551
41    4739577523
42    1983894936
43    4394418962
44    6031320708
45    8133904005
46    5272014914
47    5139003610
48    4360634054
49    7520272603
Name: Phone, dtype: object

## Splitting Text 

A powerful tool we can use to split text into columns is use the `str.split()` function. We provide a pattern that can be a separator (like commas `,`) or a full-on regular expression pattern. 

Here is how we can separate out the email domains into separate columns. We can then rename these columns and append them back to our dataframe. 

In [77]:
df['Email'].str.split("@", expand=True, regex=False)

,0,1
0,tony.reed,randatmail.com
1,v.hawkins,randatmail.com
2,e.anderson,randatmail.com
3,m.mitchell,randatmail.com
4,m.riley,randatmail.com
5,s.richards,randatmail.com
6,m.holmes,randatmail.com
7,m.murphy,outlook.com
8,g.allen,randatmail.com
9,c.kelley,randatmail.com


When you use regular expression features like look-aheads, it opens up more powerful splitting capabilities based on surrounding characters. This is beyond the scope of this notebook. 

## Exercise

Complete the code below by replacing the question mark `?`. Replace it with a regular expression operation to identify records that are missing a street number in the dataframe.

In [81]:
import pandas as pd

df = pd.DataFrame({
    "CUSTOMER_NAME" : ["Rex Tooling", "Prairie Construction", "Banke Logistics"],
    "STREET_ADDRESS" : ["147 Collie Way", "56 Samson Dr", "Elijah Blvd"]
})

df[? == False]

SyntaxError: invalid syntax (1442294621.py, line 8)

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [83]:
import pandas as pd

df = pd.DataFrame({
    "CUSTOMER_NAME" : ["Rex Tooling", "Prairie Construction", "Banke Logistics"],
    "STREET_ADDRESS" : ["147 Collie Way", "56 Samson Dr", "Elijah Blvd"]
})

df[df["STREET_ADDRESS"].str.fullmatch("[0-9]+ [A-Za-z0-9]+ (Way|Blvd|Dr|St)") == False]

,CUSTOMER_NAME,STREET_ADDRESS
2,Banke Logistics,Elijah Blvd
